In [107]:
# We implement a simple version of GPT 2 using the transformer architecture, following attention is all you need paper
# In our case, we build a character level language model, which will be trained on tinyshakespeare dataset
import torch 

In [108]:
# Open data
with open('tinyshakespeare.txt', 'r', encoding = 'utf-8') as f:
    text = f.read() 

In [109]:
print("length of data: ", len(text))

length of data:  1115394


In [110]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [111]:
chars = sorted(list(set(text)))
vocab_size = len(chars) 
print("number of unique characters: ", vocab_size)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
print(itos)

number of unique characters:  65
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [112]:
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [113]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [114]:
# Split in to train anv validation

n = int(0.9 * len(data))
train_data = data[:n]
val_data= data[n:]


In [115]:
block_size = 8
batch_size = 4

In [116]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

# # visualize a batch
xb, yb = get_batch('train')
# print('inputs:')
# print(xb.shape, xb.dtype)
# print(xb)
# print('targets:')
# print(yb.shape, yb.dtype)
# print(yb)

# print("-------")
# for b in range(batch_size):
#     for t in range(block_size):
#         context = xb[b, :t+1]
#         target = yb[b, t]
#         print(f'When input is {context.tolist()} target is {target.item()}')

In [117]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensors

        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # Becomes (B,C)
            probs = F.softmax(logits, dim=-1)  # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx


m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out[0].shape)
print(out[1])
idx = torch.zeros((1,1), dtype=torch.long)

print(decode(m.generate(idx, 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [119]:
# Training loop 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [123]:
batch_size = 32
for steps in range(100000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if steps % 100 == 0:
        print(f'step: {steps}, loss: {loss.item()}')

step: 0, loss: 2.571192741394043
step: 100, loss: 2.4889583587646484
step: 200, loss: 2.544487714767456
step: 300, loss: 2.5225296020507812
step: 400, loss: 2.5682666301727295
step: 500, loss: 2.3110439777374268
step: 600, loss: 2.4499902725219727
step: 700, loss: 2.5348362922668457
step: 800, loss: 2.47645902633667
step: 900, loss: 2.4677135944366455
step: 1000, loss: 2.4076132774353027
step: 1100, loss: 2.576707124710083
step: 1200, loss: 2.429227590560913
step: 1300, loss: 2.5243372917175293
step: 1400, loss: 2.399763345718384
step: 1500, loss: 2.4520907402038574
step: 1600, loss: 2.466071844100952
step: 1700, loss: 2.38797664642334
step: 1800, loss: 2.599693536758423
step: 1900, loss: 2.4532437324523926
step: 2000, loss: 2.4377825260162354
step: 2100, loss: 2.4375531673431396
step: 2200, loss: 2.3875503540039062
step: 2300, loss: 2.6257407665252686
step: 2400, loss: 2.3347315788269043
step: 2500, loss: 2.494647741317749
step: 2600, loss: 2.5356574058532715
step: 2700, loss: 2.46797

In [124]:
print(decode(m.generate(idx, 500)[0].tolist()))


K:
Thel BExckngl e wante buce be g scheeaneanefr qupore.
Hiceshoou dur lllowisat,
Brull d age arere we, that,
Wh atthe I

Hor thal, thing t d.
K:
Ann fr:

D:
SINor WAs d berenen! a ito tithaferot! ortoowilor fethouse;
ave uf e ldydofomangou: fodiereindsllly RCKim
TES:

Twhano e h s'dr y towatin bit do, tthinatathaid athr:
DULAGBy a bl:
Bour sar hos s ten s afeshe?
Tw, thy lar d won l ve t, mbe ce he d at ICO:

Thangllsl s aveloutr usulde prded frdsatheatorin an Drspo pomaced athid my ourtat chen
